In [1]:
import csv
import math
import matplotlib.pyplot as plt
from ipyfilechooser import FileChooser
from ipywidgets import Button, VBox, Output, Dropdown, HTML, HBox
from IPython.display import display, FileLink

# ---- constants ----
EPS = 1e-9
MPS_TO_KMH = 3.6
MPS_TO_MPH = 2.23694
M_TO_KM = 1.0 / 1000.0
M_TO_MI = 1.0 / 1609.344


def compute_scale_factors(ref_lat_deg):
    ref_lat_rad = math.radians(ref_lat_deg)
    Mphi = 111132 - 559 * math.cos(2 * ref_lat_rad) + 1.175 * math.cos(4 * ref_lat_rad)
    Mlambda = 111320 * math.cos(ref_lat_rad) - 94 * math.cos(3 * ref_lat_rad)
    return Mphi, Mlambda


def process_csv(input_file, lat_col, lon_col, time_col):
    with open(input_file, newline='', encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        headers = [h.strip() for h in reader.fieldnames]
        rows = [{k.strip(): v for k, v in r.items()} for r in reader]

    if len(rows) < 2:
        print("Not enough rows to compute speeds.")
        return

    ref_lat = float(rows[0][lat_col])
    Mphi, Mlambda = compute_scale_factors(ref_lat)

    # initialize columns
    for row in rows:
        for col in [
            "relative_time",
            "segment_distance_m", "segment_distance_km", "segment_distance_mi",
            "cumulative_distance_m", "cumulative_distance_km", "cumulative_distance_mi",
            "speed_mps", "speed_kmh", "speed_mph"
        ]:
            row[col] = ""

    t0 = float(rows[0][time_col])
    cumulative_distance_m = 0.0
    times, speeds_mph = [], []

    for i in range(len(rows)):
        t = float(rows[i][time_col])
        relative_t = t - t0
        rows[i]["relative_time"] = f"{relative_t:.2f}"

        if i == 0:
            for key in [
                "segment_distance_m", "segment_distance_km", "segment_distance_mi",
                "cumulative_distance_m", "cumulative_distance_km", "cumulative_distance_mi",
                "speed_mps", "speed_kmh", "speed_mph"
            ]:
                rows[i][key] = "0.00"
            continue

        lat1, lon1, t1 = float(rows[i-1][lat_col]), float(rows[i-1][lon_col]), float(rows[i-1][time_col])
        lat2, lon2, t2 = float(rows[i][lat_col]), float(rows[i][lon_col]), float(rows[i][time_col])

        dlat, dlon = lat2 - lat1, lon2 - lon1
        if abs(dlat) < EPS and abs(dlon) < EPS:
            segment_distance_m = 0.0
        else:
            dx = dlon * Mlambda
            dy = dlat * Mphi
            segment_distance_m = math.sqrt(dx**2 + dy**2)

        cumulative_distance_m += segment_distance_m
        dt = t2 - t1
        if dt > 0:
            speed_mps = segment_distance_m / dt
            speed_kmh = speed_mps * MPS_TO_KMH
            speed_mph = speed_mps * MPS_TO_MPH
        else:
            speed_mps = speed_kmh = speed_mph = float("nan")

        rows[i]["segment_distance_m"] = f"{segment_distance_m:.2f}"
        rows[i]["segment_distance_km"] = f"{segment_distance_m * M_TO_KM:.3f}"
        rows[i]["segment_distance_mi"] = f"{segment_distance_m * M_TO_MI:.3f}"
        rows[i]["cumulative_distance_m"] = f"{cumulative_distance_m:.2f}"
        rows[i]["cumulative_distance_km"] = f"{cumulative_distance_m * M_TO_KM:.3f}"
        rows[i]["cumulative_distance_mi"] = f"{cumulative_distance_m * M_TO_MI:.3f}"
        rows[i]["speed_mps"] = f"{speed_mps:.2f}"
        rows[i]["speed_kmh"] = f"{speed_kmh:.2f}"
        rows[i]["speed_mph"] = f"{speed_mph:.2f}"

        times.append(relative_t)
        speeds_mph.append(speed_mph)

    new_headers = headers + [
        "relative_time",
        "segment_distance_m", "segment_distance_km", "segment_distance_mi",
        "cumulative_distance_m", "cumulative_distance_km", "cumulative_distance_mi",
        "speed_mps", "speed_kmh", "speed_mph"
    ]

    with open("results.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=new_headers)
        writer.writeheader()
        writer.writerows(rows)

    


    print("✅ Results saved to results.csv")
    FileLink("results.csv")

    plt.figure(figsize=(10, 5))
    plt.plot(times, speeds_mph, marker="o", linestyle="-")
    plt.xlabel("Relative Time (s)")
    plt.ylabel("Speed (mph)")
    plt.title("Speed vs Relative Time")
    plt.grid(True)
    plt.show()

# ---- UI setup ----
def run_with_ui():
    output = Output()
    fc = FileChooser('.', filter_pattern='*.csv')
    button = Button(description="🚀 Process CSV", button_style='success')

    def on_click(b):
        with output:
            output.clear_output()
            if not fc.selected:
                print("⚠️ Please select a CSV file first.")
                return

            filename = fc.selected
            print(f"Selected file: {filename}")

            # Read headers for column selection
            with open(filename, newline='', encoding="utf-8-sig") as f:
                reader = csv.DictReader(f)
                headers = [h.strip() for h in reader.fieldnames]

            # Create dropdown widgets
            lat_dropdown = Dropdown(options=headers, description='Latitude:')
            lon_dropdown = Dropdown(options=headers, description='Longitude:')
            time_dropdown = Dropdown(options=headers, description='Timestamp:')
            
            process_btn = Button(description="✅ Process", button_style='success')
            
            def process_data(b):
                with output:
                    output.clear_output()
                    print(f"📁 Processing: {filename}")
                    print(f"📍 Using columns - Lat: {lat_dropdown.value}, Lon: {lon_dropdown.value}, Time: {time_dropdown.value}")
                    print("---")
                    
                    try:
                        process_csv(filename, lat_dropdown.value, lon_dropdown.value, time_dropdown.value)
                        print("✅ Processing completed!")
                    except Exception as e:
                        print(f"❌ Error: {e}")
            
            process_btn.on_click(process_data)
            
            # Display widgets
            display(VBox([
                HTML("<h4>Select Columns:</h4>"),
                HBox([lat_dropdown, lon_dropdown, time_dropdown]),
                process_btn
            ]))

    button.on_click(on_click)
    display(VBox([fc, button, output]))


# ---- Run interactive UI ----
run_with_ui()

ModuleNotFoundError: No module named 'ipyfilechooser'